<a href="https://colab.research.google.com/github/hschilling/google_colab/blob/master/pandas_in_openmdao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openmdao

     |████████████████████████████████| 4.4MB 2.7MB/s 
  Created wheel for openmdao: filename=openmdao-3.0.0-cp36-none-any.whl size=4626730 sha256=ccd079c1663d72b8e09468643a7a41f1142eb7192e5b51f82ca7b85b6f0c3eff
  Stored in directory: /root/.cache/pip/wheels/ad/8f/77/c58ef6cab51fe796172ce215f1c66319164def6edf11c43b88
  Created wheel for pyDOE2: filename=pyDOE2-1.3.0-cp36-none-any.whl size=25519 sha256=8dece769cfb2f177eaf80ddb50a2603b6fd89f3f36dc01838faed4feb289334b
  Stored in directory: /root/.cache/pip/wheels/60/88/1d/b334ee00dd83e82d111ec56c3be91573d335c93870698037f1
Successfully built openmdao pyDOE2


In [2]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

driver.add_recorder(om.SqliteRecorder("cases.sql"))

driver.recording_options['includes'] = []
driver.recording_options['record_objectives'] = True
driver.recording_options['record_constraints'] = True
driver.recording_options['record_desvars'] = True

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")


Unable to import mpi4py. Parallel processing unavailable.


In [14]:
from collections import defaultdict
case = cr.get_case(0)
outputs = case.list_outputs(out_stream=None)
print(sorted([out[0] for out in outputs]))
varnames = [out[0] for out in outputs]
data = defaultdict(list)
for i in range(len(cr.list_cases())):
  case = cr.get_case(i)
  for varname in varnames:
    data[varname].append(case[varname])

 

['con_cmp1.con1', 'con_cmp2.con2', 'obj_cmp.obj', 'px.x', 'pz.z']


In [0]:
case = cr.get_case(0)
outputs = case.list_outputs(out_stream=None)
varnames = [out[0] for out in outputs]
data = {}
for i in range(len(cr.list_cases())):
  case = cr.get_case(i)
  for varname in varnames:
    if varname not in data:
        data[varname] = []
    data[varname].append(case[varname][0])



In [41]:
data

{'con_cmp1.con1': [-22.42830236987513,
  -22.428302369877738,
  -4.574337391558954,
  -0.5507241567968015,
  -0.020433821267987984,
  -3.282107686741753e-05,
  -8.380540705843487e-11],
 'con_cmp2.con2': [-11.94151184938868,
  -11.941511849388423,
  -17.52922772624373,
  -19.947351998016885,
  -20.233245833384178,
  -20.244703769823385,
  -20.2447222330272],
 'obj_cmp.obj': [28.588308165031194,
  28.5883081650338,
  8.56152842495788,
  3.728100457981922,
  3.2035608292750446,
  3.1834263407931056,
  3.183393951723336],
 'px.x': [1.0,
  1.0,
  1.2878587085651816e-14,
  0.0,
  0.0,
  1.1604064831183036e-15,
  0.0],
 'pz.z': [5.0,
  5.0,
  2.864061598967984,
  2.1263240009913784,
  1.9833770833078666,
  1.977648115088326,
  1.9776388834861036]}

In [0]:
import pandas as pd

In [42]:
df = pd.DataFrame(data)
df.drop(['pz.z'], axis=1)

,px.x,obj_cmp.obj,con_cmp1.con1,con_cmp2.con2
0,1.000000e+00,28.588308,-2.242830e+01,-11.941512
1,1.000000e+00,28.588308,-2.242830e+01,-11.941512
2,1.287859e-14,8.561528,-4.574337e+00,-17.529228
3,0.000000e+00,3.728100,-5.507242e-01,-19.947352
4,0.000000e+00,3.203561,-2.043382e-02,-20.233246
5,1.160406e-15,3.183426,-3.282108e-05,-20.244704
6,0.000000e+00,3.183394,-8.380541e-11,-20.244722


In [31]:
df[['con_cmp1.con1']]

,con_cmp1.con1
0,[-22.42830236987513]
1,[-22.428302369877738]
2,[-4.574337391558954]
3,[-0.5507241567968015]
4,[-0.020433821267987984]
5,[-3.282107686741753e-05]
6,[-8.380540705843487e-11]


In [43]:
import cufflinks as cf
fig = df.iplot(asFigure=True)
fig.show()


In [24]:
len(cr.list_cases())

7

In [33]:
df.describe()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


TypeError: ignored

...

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


,pz.z,px.x,obj_cmp.obj,con_cmp1.con1,con_cmp2.con2
count,7,7,7,7,7
unique,7,4,7,7,7
top,"[1.977648115088326, 2.3627206883882793e-14]",[0.0],[3.1834263407931056],[-3.282107686741753e-05],[-20.244703769823385]
freq,1,3,1,1,1


In [34]:
df.head()

,pz.z,px.x,obj_cmp.obj,con_cmp1.con1,con_cmp2.con2
0,"[5.0, 2.0]",[1.0],[28.588308165031194],[-22.42830236987513],[-11.94151184938868]
1,"[5.0, 2.0]",[1.0],[28.5883081650338],[-22.428302369877738],[-11.941511849388423]
2,"[2.864061598967984, 0.8256430036305078]",[1.2878587085651816e-14],[8.56152842495788],[-4.574337391558954],[-17.52922772624373]
3,"[2.1263240009913784, 1.6653345369377348e-15]",[0.0],[3.728100457981922],[-0.5507241567968015],[-19.947351998016885]
4,"[1.9833770833078666, 0.0]",[0.0],[3.2035608292750446],[-0.020433821267987984],[-20.233245833384178]
